In [69]:
import pandas as pd 
import glob 
from typing import Optional
import os 
import re
import numpy as np
from datetime import datetime
from thefuzz import fuzz, process
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root not in sys.path:
    sys.path.append(project_root)
    
from utils.getLatestFiles import get_latest_cleaned_matches

In [70]:
CLEANED_MATCHES_DIR = "../Data/Processed/Matches/"
CLEANED_MATCHES_SUFFIX = "_cleaned_matches.csv"
CLEANED_MATCHES_REGEX = rf"^\d{{8}}{re.escape(CLEANED_MATCHES_SUFFIX)}$"
START_DROP_COLS = [
    "A_rawGameScores",
    "A_rawOverallScore",
    "B_rawGameScores",
    "B_rawOverallScore",
    "actionType",
    "awayNestedGameScores",
    "bestOf",
    "calcBestOf",
    "calcNestedGameScores",
    "calcNestedOverallScores",   
    "currentGameNumber",
    "gameScoreConsistent",
    "gameScoreFlags",
    "homeNestedGameScores",
    "scoreConsistent",
    "tableNumber",
    'calculatedGameScoreWinner'
    
]

now_date = datetime.now()
date_string = now_date.strftime("%Y%m%d") 

MASTER_MATCHES_DIR = "../Data/Master/Matches"
os.makedirs(MASTER_MATCHES_DIR, exist_ok=True)
MASTER_MATCHES_SUFFIX = "_master_matches.csv"
MASTER_MATCHES_OUTPUT_PATH = os.path.join(MASTER_MATCHES_DIR,f"{date_string}{MASTER_MATCHES_SUFFIX}")



In [71]:
def get_latest_cleaned_matches(cleaned_matches_dir:str, cleaned_matches_regex) -> Optional[pd.DataFrame]:
    """
    Parses specified directory for cleaned_matches_files in format yyyy_mm_dd. 
    Attempts to read latest file in this format. 

    Args:
        directory (str): The folder where the clened_matches_files are stored 
        filename_pattern (str): The pattern to match 

    Returns:
        Tuple[pd.DataFrame,Optional]: returns DF with data if available or blank df if data unavailable
    """
    if not os.path.isdir(cleaned_matches_dir):
        print (f"❌{cleaned_matches_dir} does not exist as a directory")
            
        
    # Get csv files in 
    files = glob.glob(f"{cleaned_matches_dir}/*.csv")


    cleaned_matches_files = []


    for file in files:
        filename = os.path.basename(file)   
         
        
        if re.match(cleaned_matches_regex,filename):
            cleaned_matches_files.append(file)
            print(f"✅ Found cleaned_matches file: {file}")

    if not cleaned_matches_files:
        print(f"❌ No existing cleaned_matches files in format: {cleaned_matches_regex} in {cleaned_matches_dir}")
        None 
    cleaned_matches_files.sort()    
    latest_cleaned_matches = cleaned_matches_files[-1]

    try: 
        latest_cleaned_matches_df = pd.read_csv(latest_cleaned_matches)
        print(f"✅ {len(latest_cleaned_matches_df)} matches found in latest cleaned_matches: {latest_cleaned_matches} ")
        return latest_cleaned_matches_df
        
    except Exception as e:
        print (f"❌ Error reading lastest cleaned_matches, {latest_cleaned_matches}: {e}")
        None 

In [72]:
cleaned_matches_df = get_latest_cleaned_matches(CLEANED_MATCHES_DIR, CLEANED_MATCHES_REGEX)
cleaned_matches_df.drop(columns=START_DROP_COLS, inplace=True, errors='ignore')
cleaned_matches_df.rename(columns={'reconciledOverallScoreWinner (more reliable)':"Winner"}, inplace=True) 


✅ Found cleaned_matches file: ../Data/Processed/Matches/20251114_cleaned_matches.csv
✅ Found cleaned_matches file: ../Data/Processed/Matches/20251125_cleaned_matches.csv
✅ Found cleaned_matches file: ../Data/Processed/Matches/20251119_cleaned_matches.csv
✅ Found cleaned_matches file: ../Data/Processed/Matches/20251115_cleaned_matches.csv
✅ Found cleaned_matches file: ../Data/Processed/Matches/20251111_cleaned_matches.csv
✅ Found cleaned_matches file: ../Data/Processed/Matches/20251108_cleaned_matches.csv
✅ Found cleaned_matches file: ../Data/Processed/Matches/20251110_cleaned_matches.csv
✅ Found cleaned_matches file: ../Data/Processed/Matches/20251117_cleaned_matches.csv
✅ Found cleaned_matches file: ../Data/Processed/Matches/20251120_cleaned_matches.csv
✅ Found cleaned_matches file: ../Data/Processed/Matches/20251112_cleaned_matches.csv
✅ 23945 matches found in latest cleaned_matches: ../Data/Processed/Matches/20251125_cleaned_matches.csv 


/tmp/ipykernel_610210/4214941264.py:39: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  latest_cleaned_matches_df = pd.read_csv(latest_cleaned_matches)


In [73]:
"""
Shortened code for mapping home and away to winner and loser 
dependent on who won - thus removing home and away terms
MAY need to restore home and away terms later if further analysis is done from other match data sources
e.g match logs 
"""
is_away_winner = cleaned_matches_df["Winner"] == "away"

column_map = {
    
    'winnerName':    ('homePlayer',         'awayPlayer'),
    'winnerId':      ('homeCompetitorId',   'awayCompetitorId'),
    'winnerCountry': ('homeCompetitorOrg',  'awayCompetitorOrg'),
    
    
    'loserName':     ('awayPlayer',         'homePlayer'),
    'loserId':       ('awayCompetitorId',   'homeCompetitorId'),
    'loserCountry':  ('awayCompetitorOrg',  'homeCompetitorOrg'),
}

for new_col, (away_case, home_case) in column_map.items():
    cleaned_matches_df[new_col] = np.where(
        is_away_winner,
        cleaned_matches_df[home_case],  # Value if True
        cleaned_matches_df[away_case]  # Value if False
    )

cols_to_drop = ["homePlayer", "awayPlayer", "homeCompetitorId", "awayCompetitorId", "homeCompetitorOrg", "awayCompetitorOrg"]
integer_columns = ["trueBestOf", "winnerId", "loserId"]

cleaned_matches_df.drop(columns=cols_to_drop, inplace=True, errors='ignore')
for col in integer_columns:
    cleaned_matches_df[col] = cleaned_matches_df[col].astype("Int64")


In [74]:
cleaned_matches_df.rename(columns={
    "reconciledOverallScore": "overallScore",
    "reconciledGameScore": "gameScore",
    "trueBestOf": "bestOf",
}, inplace=True)

In [75]:
score_splits = cleaned_matches_df["overallScore"].str.split("-", expand=True)
home_score_numeric = pd.to_numeric(score_splits[0], errors='coerce')
away_score_numeric = pd.to_numeric(score_splits[1], errors='coerce')
winner_correct = (home_score_numeric >= away_score_numeric)

In [76]:
def switch_overall_score(score_str):
    """
    Safely switches a single score string like "2-4" to "4-2".
    Uses regex to be robust and handle DNF strings (like 'WO') safely.
    """
    if pd.isna(score_str):
        return pd.NA
    return re.sub(r'(\d+)-(\d+)', r'\2-\1', str(score_str))


def switch_game_score(score_str):
    """
    Switches a comma-separated list of scores.
    (This is your function, slightly hardened with a try-except block)
    """
    if pd.isna(score_str) or not score_str:
        return pd.NA
    try:
        
        return re.sub(r'(\d+)-(\d+)', r'\2-\1', str(score_str))
    except Exception:
        return pd.NA 
    
rows_to_switch = (winner_correct == False)
print(f"--- ⚙️ Standardizing Scores to 'Winner-First' Format ---")
print(f"Identifying {rows_to_switch.sum()} rows to switch...")
cleaned_matches_df.loc[rows_to_switch, 'overallScore'] =  cleaned_matches_df.loc[rows_to_switch, 'overallScore'].apply(switch_overall_score)

# Apply the switch to 'gameScore'
cleaned_matches_df.loc[rows_to_switch, 'gameScore'] = cleaned_matches_df.loc[rows_to_switch, 'gameScore'].apply(switch_game_score)

print("✅ Scores successfully standardized.")

--- ⚙️ Standardizing Scores to 'Winner-First' Format ---
Identifying 11596 rows to switch...
✅ Scores successfully standardized.


In [77]:
score_splits = cleaned_matches_df["overallScore"].str.split("-", expand=True)
home_score_numeric = pd.to_numeric(score_splits[0], errors='coerce')
away_score_numeric = pd.to_numeric(score_splits[1], errors='coerce')
winner_correct = (home_score_numeric >= away_score_numeric)

In [78]:
#final amendments 
cleaned_matches_df.loc[~winner_correct, "Winner"] = pd.NA

In [79]:
score_parts = cleaned_matches_df['overallScore'].str.split('-', expand=True)
winner_sets = pd.to_numeric(score_parts[0], errors='coerce').astype('Int64')
loser_sets = pd.to_numeric(score_parts[1], errors='coerce').astype('Int64')
cleaned_matches_df['winnerSets'] = winner_sets
cleaned_matches_df['loserSets'] = loser_sets
cleaned_matches_df['totalSets'] = cleaned_matches_df['winnerSets'] + cleaned_matches_df['loserSets']
cleaned_matches_df['isStraightSets'] = (cleaned_matches_df['loserSets'] == 0)
cleaned_matches_df["isDecider"] = (
    cleaned_matches_df['totalSets'] == cleaned_matches_df['bestOf']
)



In [80]:
import pandas as pd
import numpy as np

def engineer_game_score_stats(row):
    game_score = row["gameScore"]
    
    output_cols = [
        "winnerTotalPoints", "loserTotalPoints", "totalPoints", 
        "winnerPointsRatio", "loserPointsRatio", "winnerMaxScore", 
        "loserMaxScore", "maxScore", "numberDeuceGames", 
        "winnerDeuceWon", "loserDeuceWon", "winnerPointsDifference",
        "pointsPerSet","winnerDroppedFirstSet",
        "winner0Wins", "winner1Wins", "winner4Wins",
        "loser0Wins", "loser1Wins", "loser4Wins",
        "winnerAvgWinningMargin", "loserAvgWinningMargin",
        "comebackBy2", "comebackBy3",
        "winnerAvgPointsPerSet", "loserAvgPointsPerSet"
    ]
    
    # 1. Handle missing/empty input
    if pd.isna(game_score) or not str(game_score).strip():
        return pd.Series(pd.NA, index=output_cols)
        
    try:
        # 2. Split the string (e.g., "11-9,9-11")
        game_scores_split = [s for s in str(game_score).split(",") if s.strip()]

        if not game_scores_split:
            return pd.Series(pd.NA, index=output_cols)
            
        # 3. CRITICAL FIX: Convert to integers immediately
        # We strip whitespace and force int conversion here.
        winner_scores = [int(score.split("-")[0].strip()) for score in game_scores_split]
        loser_scores = [int(score.split("-")[1].strip()) for score in game_scores_split]
        
        # --- Basic Stats ---
        winner_total = sum(winner_scores)
        loser_total = sum(loser_scores)
        total_points = winner_total + loser_total
        total_sets = len(game_scores_split)

        if total_points > 0:
            winner_ratio = round(winner_total / total_points, 3)
            loser_ratio = round(loser_total / total_points, 3)
        else:
            winner_ratio = np.nan
            loser_ratio = np.nan

        # Safety for empty lists (though logic above should prevent it)
        winner_max = max(winner_scores) if winner_scores else 0
        loser_max = max(loser_scores) if loser_scores else 0
        max_score = max(winner_max, loser_max)
        
        # --- Deuce Logic ---
        num_deuce = sum(1 for w, l in zip(winner_scores, loser_scores) if w >= 10 and l >= 10)
        winner_deuce_won = sum(1 for w, l in zip(winner_scores, loser_scores) if w > l and w >= 12)
        loser_deuce_won = sum(1 for w, l in zip(winner_scores, loser_scores) if l > w and l >= 12)

        winner_points_difference = winner_total - loser_total
        points_per_set = round(total_points / total_sets, 2)
        winner_avg_points_per_set = round(winner_total / total_sets, 2)
        loser_avg_points_per_set = round(loser_total / total_sets, 2)

        # --- Advanced Logic ---
        winner_dropped_first_set = (winner_scores[0] < loser_scores[0])
        
        winner_wins_to_love_count = 0
        winner_wins_to_1_count = 0
        winner_wins_to_4_count = 0
        loser_wins_to_love_count = 0
        loser_wins_to_1_count = 0
        loser_wins_to_4_count = 0

        win_margins = []
        lose_margins = []
        
        winner_set_score = 0
        loser_set_score = 0
        comeback_by_2 = False
        comeback_by_3 = False
        
        for w_pts, l_pts in zip(winner_scores, loser_scores):
            # Note: w_pts and l_pts are guaranteed ints now
            if w_pts > l_pts:
                winner_set_score += 1
                win_margins.append(w_pts - l_pts)
                if l_pts == 0:
                    winner_wins_to_love_count += 1
                if l_pts == 1:
                    winner_wins_to_1_count += 1
                if l_pts <= 4:
                    winner_wins_to_4_count += 1
            elif l_pts > w_pts:
                loser_set_score += 1
                lose_margins.append(l_pts - w_pts)
                if w_pts == 0:
                    loser_wins_to_love_count += 1
                if w_pts == 1:
                    loser_wins_to_1_count += 1
                if w_pts <= 4:
                    loser_wins_to_4_count += 1
            
            deficit = loser_set_score - winner_set_score
            if deficit == 2:
                comeback_by_2 = True
            if deficit == 3:
                comeback_by_3 = True
        
        winner_average_win_margin = 0.0
        if win_margins:
            winner_average_win_margin = round(sum(win_margins) / len(win_margins), 2)
            
        loser_average_win_margin = 0.0
        if lose_margins:
            loser_average_win_margin = round(sum(lose_margins) / len(lose_margins), 2)

        return pd.Series({
            "winnerTotalPoints": winner_total, 
            "loserTotalPoints": loser_total,
            "totalPoints": total_points,
            "winnerPointsRatio": winner_ratio,
            "loserPointsRatio": loser_ratio,
            "winnerMaxScore": winner_max,
            "loserMaxScore": loser_max,
            "maxScore": max_score,
            "numberDeuceGames": num_deuce,
            "winnerDeuceWon": winner_deuce_won,
            "loserDeuceWon": loser_deuce_won,
            "winnerPointsDifference": winner_points_difference,
            "pointsPerSet": points_per_set,          
            "winnerDroppedFirstSet": winner_dropped_first_set,
            "winner0Wins": winner_wins_to_love_count,
            "winner1Wins": winner_wins_to_1_count,
            "winner4Wins" : winner_wins_to_4_count,
            "loser0Wins": loser_wins_to_love_count,
            "loser1Wins": loser_wins_to_1_count,
            "loser4Wins" : loser_wins_to_4_count,
            "winnerAvgWinningMargin": winner_average_win_margin,
            "loserAvgWinningMargin": loser_average_win_margin,
            "comebackBy2": comeback_by_2,
            "comebackBy3": comeback_by_3,
            "winnerAvgPointsPerSet": winner_avg_points_per_set,
            "loserAvgPointsPerSet": loser_avg_points_per_set
        })
        
    except (ValueError, IndexError, TypeError, AttributeError) as e:
        print(f"Error: {e}") # Uncomment for debugging
        return pd.Series(pd.NA, index=output_cols)

In [81]:
new_cols=  cleaned_matches_df.apply(engineer_game_score_stats, axis=1, result_type='expand')

In [82]:
new_cols

,winnerTotalPoints,loserTotalPoints,totalPoints,winnerPointsRatio,loserPointsRatio,winnerMaxScore,loserMaxScore,maxScore,numberDeuceGames,winnerDeuceWon,...,winner4Wins,loser0Wins,loser1Wins,loser4Wins,winnerAvgWinningMargin,loserAvgWinningMargin,comebackBy2,comebackBy3,winnerAvgPointsPerSet,loserAvgPointsPerSet
0,36,29,65,0.554,0.446,13,11,13,2,2,...,0,0,0,0,2.33,0.0,False,False,12.0,9.67
1,34,17,51,0.667,0.333,12,10,12,1,1,...,2,0,0,0,5.67,0.0,False,False,11.33,5.67
2,33,26,59,0.559,0.441,11,9,11,0,0,...,0,0,0,0,2.33,0.0,False,False,11.0,8.67
3,37,28,65,0.569,0.431,15,13,15,1,1,...,0,0,0,0,3.0,0.0,False,False,12.33,9.33
4,36,25,61,0.59,0.41,14,12,14,1,1,...,0,0,0,0,3.67,0.0,False,False,12.0,8.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23940,41,35,76,0.539,0.461,15,13,15,1,1,...,1,0,0,1,4.33,7.0,False,False,10.25,8.75
23941,42,31,73,0.575,0.425,12,11,12,1,1,...,1,0,0,0,4.67,3.0,False,False,10.5,7.75
23942,35,24,59,0.593,0.407,13,11,13,1,1,...,0,0,0,0,3.67,0.0,False,False,11.67,8.0
23943,33,20,53,0.623,0.377,11,9,11,0,0,...,1,0,0,0,4.33,0.0,False,False,11.0,6.67


In [83]:
cleaned_matches_df = pd.concat([cleaned_matches_df, new_cols], axis=1)

In [84]:
tie_filter = cleaned_matches_df["Winner"]=="tie"
len(cleaned_matches_df[tie_filter])
cleaned_matches_df[tie_filter][["EventName","winnerName", "loserName", "Winner","overallScore","gameScore"]]


,EventName,winnerName,loserName,Winner,overallScore,gameScore
14188,ITTF MEN'S AND WOMEN'S WORLD CUP MACAO 2024 PR...,QIU Dang,SALEH Ahmed,tie,2-2,"8-11,11-3,11-7,6-11"
14201,ITTF MEN'S AND WOMEN'S WORLD CUP MACAO 2024 PR...,LIN Gaoyuan,PUCAR Tomislav,tie,2-2,"12-10,11-7,11-13,8-11"
14214,ITTF MEN'S AND WOMEN'S WORLD CUP MACAO 2024 PR...,ITO Mima,SAWETTABUT Suthasini,tie,2-2,"14-12,9-11,3-11,11-6"
14215,ITTF MEN'S AND WOMEN'S WORLD CUP MACAO 2024 PR...,WANG Chuqin,AN Jaehyun,tie,2-2,"11-9,11-7,9-11,8-11"
14230,ITTF MEN'S AND WOMEN'S WORLD CUP MACAO 2024 PR...,LEBRUN Alexis,MINO Alberto,tie,2-2,"3-11,11-9,9-11,11-4"
14235,ITTF MEN'S AND WOMEN'S WORLD CUP MACAO 2024 PR...,WANG Amy,SAWETTABUT Suthasini,tie,2-2,"9-11,11-7,4-11,11-7"
14240,ITTF MEN'S AND WOMEN'S WORLD CUP MACAO 2024 PR...,GERASSIMENKO Kirill,AN Jaehyun,tie,2-2,"11-7,9-11,1-11,12-10"
14250,ITTF MEN'S AND WOMEN'S WORLD CUP MACAO 2024 PR...,KALLBERG Anton,LIM Jonghoon,tie,2-2,"6-11,6-11,11-9,14-12"
14251,ITTF MEN'S AND WOMEN'S WORLD CUP MACAO 2024 PR...,WANG Yidi,SHAN Xiaona,tie,2-2,"11-7,11-4,9-11,9-11"
14255,ITTF MEN'S AND WOMEN'S WORLD CUP MACAO 2024 PR...,HARIMOTO Tomokazu,GROTH Jonathan,tie,2-2,"7-11,6-11,11-6,11-5"


In [85]:
final_desc_filter = cleaned_matches_df["subEventDescription"].str.contains(" Final",case=False)
final_code_filter = cleaned_matches_df["documentCode"].str.contains("-FNL",case=False)

final_mismatch_1 = final_desc_filter & ~final_code_filter
final_mismatch_2 = ~final_desc_filter & final_code_filter

final_mismatch_filter = final_mismatch_1 | final_mismatch_2
cleaned_matches_df[final_mismatch_filter]

,eventId,documentCode,subEventName,subEventDescription,venueName,tableName,dnf,ttrReview,serverNext,duration (unreliable),...,winner4Wins,loser0Wins,loser1Wins,loser4Wins,winnerAvgWinningMargin,loserAvgWinningMargin,comebackBy2,comebackBy3,winnerAvgPointsPerSet,loserAvgPointsPerSet
545,2345,TTEWSINGLES-----------FNL-000200----------,Women's Singles,Women's Singles Bronze Medal Match,Tokyo Metropolitan Gym,Table 1,False,NaN,NaN,0:47,...,0,0,0,0,4.0,5.0,False,False,10.0,7.8
546,2345,TTEWSINGLES-----------FNL-000100----------,Women's Singles,Women's Singles Gold Medal Match,Tokyo Metropolitan Gym,Table 1,False,NaN,NaN,1:05,...,2,0,0,0,5.25,4.0,False,False,9.67,7.5
547,2345,TTEMSINGLES-----------FNL-000200----------,Men's Singles,Men's Singles Bronze Medal Match,Tokyo Metropolitan Gym,Table 1,False,NaN,NaN,1:06,...,1,0,0,1,3.75,4.67,False,False,9.86,9.71
548,2345,TTEMSINGLES-----------FNL-000100----------,Men's Singles,Men's Singles Gold Medal Match,Tokyo Metropolitan Gym,Table 1,False,NaN,NaN,1:04,...,1,0,0,1,4.0,5.0,False,False,9.5,8.5
15781,2603,TTEMSINGLES-----------FNL-000200----------,Men's Singles,Men's Singles Bronze Medal Match,South Paris Arena 4,Table 1,False,False,NaN,0:31,...,0,0,0,0,4.0,0.0,False,False,11.25,7.25
15786,2603,TTEWSINGLES-----------FNL-000200----------,Women's Singles,Women's Singles Bronze Medal Match,South Paris Arena 4,Table 1,False,False,NaN,1:12,...,0,0,0,0,3.0,2.0,False,False,11.0,9.67
15822,2603,TTEMSINGLES-----------FNL-000100----------,Men's Singles,Men's Singles Gold Medal Match,South Paris Arena 4,Table 1,False,False,NaN,0:49,...,0,0,0,0,2.5,4.0,False,False,10.2,9.0
15874,2603,TTEWSINGLES-----------FNL-000100----------,Women's Singles,Women's Singles Gold Medal Match,South Paris Arena 4,Table 1,False,False,NaN,1:10,...,1,0,0,1,4.5,4.5,False,False,9.5,8.0
16929,2917,TTEMSINGLES-----------CON5000100----------,Men's Singles,Men's Singles - Consolation Final - Match 1,Auckland Table Tennis Association,Table 5,False,True,144522.0,00:00:00,...,1,0,0,0,6.33,0.0,False,False,11.0,4.67
16930,2917,TTEWSINGLES-----------CON5000100----------,Women's Singles,Women's Singles - Consolation Final - Match 1,Auckland Table Tennis Association,Table 6,False,True,200205.0,00:00:00,...,0,0,0,0,4.0,0.0,False,False,11.0,7.0


In [86]:
non_consolation_filter1 = ~cleaned_matches_df["subEventDescription"].str.contains("Consolation",case=False)
non_consolation_filter2 = ~cleaned_matches_df["documentCode"].str.contains("-CON",case=False)
non_consolation_filter = non_consolation_filter1 & non_consolation_filter2

finals_regex = " final|gold"
finals_code = "-FNL"
finals_desc_filter = cleaned_matches_df["subEventDescription"].str.contains(finals_regex,regex=True,na=False,case=False)
finals_code_filter = cleaned_matches_df["documentCode"].str.contains(finals_code,na=False,case=False)
finals_mask = finals_desc_filter & finals_code_filter & non_consolation_filter
cleaned_matches_df.loc[finals_mask, "Round"] = "Final"



bronze_regex = " bronz"
bronze_desc_filter = cleaned_matches_df["subEventDescription"].str.contains(bronze_regex,regex=True,na=False,case=False)
bronze_mask = bronze_desc_filter
cleaned_matches_df.loc[bronze_mask, "Round"] = "Bronze"



semis_regex = "semi"
semis_code = "-SFNL"
semis_desc_filter = cleaned_matches_df["subEventDescription"].str.contains(semis_regex,regex=True,na=False,case=False)
semis_code_filter = cleaned_matches_df["documentCode"].str.contains(semis_code,na=False,case=False)
semis_mask = semis_desc_filter & semis_code_filter & non_consolation_filter
cleaned_matches_df.loc[semis_mask, "Round"] = "Semifinal"


quarts_regex = "quarter"
quarts_code = "-QFNL|-8FNL"
quarts_desc_filter = cleaned_matches_df["subEventDescription"].str.contains(quarts_regex,regex=True,na=False,case=False)
quarts_code_filter = cleaned_matches_df["documentCode"].str.contains(quarts_code,na=False,case=False)
quarts_mask = quarts_desc_filter & quarts_code_filter & non_consolation_filter
cleaned_matches_df.loc[quarts_mask, "Round"] = "Quarterfinal"



for match_round in ["128", "64", "32", "16"]:  
    desc_filter = cleaned_matches_df["subEventDescription"].str.contains(match_round,regex=True,na=False,case=False) 
    code_filter = cleaned_matches_df["documentCode"].str.contains(match_round,na=False,case=False)
    mask = desc_filter | code_filter 
    cleaned_matches_df.loc[mask, "Round"] = f"R{round}"

group_regex = "group"
group_code = "gp"
group_desc_filter = cleaned_matches_df["subEventDescription"].str.contains(group_regex,regex=True,na=False,case=False)
group_code_filter = cleaned_matches_df["documentCode"].str.contains(group_code,na=False,case=False)
group_mask = group_desc_filter | group_code_filter
cleaned_matches_df.loc[group_mask, "Round"] = "Group"


for match_round in ["prelim", "qualif"]:
    round_desc_filter = desc_filter = cleaned_matches_df["subEventDescription"].str.contains(match_round,regex=True,na=False,case=False) 
    cleaned_matches_df.loc[round_desc_filter, "Round"] = match_round



cleaned_matches_df["Round"] = cleaned_matches_df["Round"].fillna("Other")

cleaned_matches_df["matchupIds"] = cleaned_matches_df.apply(
    lambda x: tuple(sorted((x["winnerId"], x["loserId"]))), 
    axis=1
)


In [87]:
cleaned_matches_df.to_csv(MASTER_MATCHES_OUTPUT_PATH,index=False)